# 用法

## 命令行

In [ ]:
pyspider [OPTIONS] COMMAND [ARGS]
'''
Options：
-C，--config FILENAME 指定配置文件名称
--logging-config TEXT 日志配置文件名称，默认：pyspider/pyspider/logging.conf
--debug 开启调试模式
--queue-maxsize INTEGER 队列的最大长度
--taskdb TEXT taskdb的数据库连接字符串，默认：sqlite
--projectdb TEXT projectdb的数据库连接字符串，默认：sqlite
--resultdb TEXT resultdb的数据库连接字符串，默认：sqlite
--message-queue TEXT 消息队列连接字符串，默认：multiprocessing.Queue
--phantomjs-proxy TEXT PhantomJS使用的代理，ip:port 的形式
--data-path TEXT 数据库存放的路径
--version pyspider的版本
--help 显示帮助信息
'''

In [ ]:
'''
pyspider WEBUI的访问认证，新建pyspider.json文件
{
    "webui":{
        "username":"root",
        "password":"123456",
        "need-auth":true
    }
}
'''
# 认证的方式进行访问，用户名为root，密码为123456
pyspider -c pyspider.json all

In [ ]:
# 单独运行Scheduler
pyspider scheduler [OPTIONS]
'''
Options：
--inqueue-limit INTEGER 任务队列的最大长度，如果满了则新的任务会被忽略
--delete-time INTEGER 设置为delete标记之前的删除时间
--active-tashks INTEGER 当前活跃任务数量配置
--loop-limit INTEGER 单轮最多调度的任务数量
--scheduler-cls TEXT Scheduler使用的类
--help 显示帮助信息
'''

In [ ]:
# 单独运行Fetcher
pyspider fetcher [OPTIONS]
'''
参数配置
Options：
--poolsize INTEGER：同事请求的个数
--proxy TEXT：使用的代理
--user-agent TEXT：使用的User-Agent
--timeout TEXT：超时时间
--fetcher-cls TEXT：Fetcher使用的类
--help：显示帮助信息
'''

In [ ]:
# 单独运行Processer
pyspider processor [OPTIONS]
'''
Options:
--processor-cls TEXT：Processor使用的类
--help：显示帮助信息
'''

In [ ]:
pyspider webui [OPTIONS]
'''
Options：
--host TEXT：运行地址
--port INTEGER：运行端口
--cdn TEXT：JS和CSS的CDN服务器
--scheduler-rpc TEXT：Scheduler的xmlrpc路径
--fetcher-rpc TEXT：Fetcher的xmlrpc路径
--max-rate FLOAT：每个项目最大的rate值
--max-burst FLOAT：每个项目最大的burst值
--username TEXT：Auth验证的用户名
--password TEXT：Auth验证的密码
--need-auth：是否需要验证
--webui-instance TEXT：运行时使用的Flask应用
--help：显示帮助信息
'''

## 项目状态

- TODO：项目刚创建还未实现时的状态
- STOP：想停止某项目的抓取，可以将项目的状态设置为STOP
- CHECKING：正在运行的项目被修改后就会改变成CHECKING状态，项目在中途出错需要调整时会遇到这种情况
- DEBUG/RUNNING：这两个状态对项目的运行没有影响，状态设置为任意一个，项目都可以运行，但是可以用两者来区分项目是否已经测试通过
- PAUSE：的那个爬取过程中出现连续多次错误时，项目会自动设置为PAUSE状态，并等待一段时间后继续爬取

## 删除项目
- pyspider中没有直接删除项目的选项
- 要删除项目：
    - 将项目设置为STOP状态
    - 将分组设置为delete
    - 等待24小时，项目会自动删除

## 爬虫的编写

- 创建一个新的爬虫进程
![](img/pyspider_2.png)

## 代码的解释

In [ ]:
'''
crawl()方法：
url - 爬取时的URL，可以定义为单个URL字符串，也可以为URL列表
callback - 回调函数，指定了该URL对应的响应内容用哪个方法来解析
age - 任务的有效时间，如果某个任务在有效时间内且已经被执行，则不会重复执行,默认有效时间为10天
priority - 爬取任务的优先级，默认是0，数值越大，对应的请求会越优先被调度
exetime - 设置定时任务，值是时间戳，默认是0，代表立即执行
retries - 重试次数，默认是3
itag - 设置判定网页是否发生变化的节点值，在爬取时会判定次当前节点是否和上次爬取到的节点相同，如果相同代表页面没有更新，就不会重复爬取
auto_recrawl - 当开启时，爬取任务在过期后会重新执行，循环时间即定义的age时间长度
method - http请求方式，默认是get
params - 定义get请求参数
data - post表单数据
files - 上传的文件，需要指定文件名
user_agent：爬取使用的User-Agent
headers：爬取时使用的headers
cookie：爬取时使用的cookies
onnect_time：初始化链接时的最长等待时间，默认是20s
timeout：抓取网页时最长等待时间，默认是120s
allow_redirects：确定是否自动处理重定向，默认是True
validate_cert：是否验证证书，对HTTPS请求有效，默认是True
proxy：爬取时使用的代理，支持用户名密码的配置
fetch_type：开启PhantomJS渲染
js_script：页面加载完毕后执行的js脚本
js_run_at：js脚本运行的位置，在页面节点开头还是结尾，默认是结尾即：document-end
js_viewport_width/js_viewport_height：js渲染时页面的窗口大小
load_images：在加载js页面时确定是否加载图片，默认为否
save：参数非常有用，可以在不同的方法之间传递参数
cancel：取消任务，如果任务是ACTIVE状态，则需要将force_update设置为True
force_update：任务处于ACTIVE状态，也会强制更新状态
'''

# age
def on_start(self):
    self.crawl('http://www.example.org/',callback=self.callback,age=10*24*60*60)

@config(age=10*24*60*60)
def callback(self):
    pass